Import Libraries

In [61]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image
import numpy as np


Define the Dataset Class

In [62]:
class MaskedUnmaskedDataset(Dataset):
    def __init__(self, masked_dir, unmasked_dir, transform=None):
        self.masked_dir = masked_dir
        self.unmasked_dir = unmasked_dir
        self.transform = transform
        self.masked_images = os.listdir(masked_dir)
        self.unmasked_images = os.listdir(unmasked_dir)
        self.images = self.masked_images + self.unmasked_images
        self.labels = [1] * len(self.masked_images) + [0] * len(self.unmasked_images)  # 1 for masked, 0 for unmasked

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        if idx < len(self.masked_images):
            img_path = os.path.join(self.masked_dir, self.masked_images[idx])
        else:
            img_path = os.path.join(self.unmasked_dir, self.unmasked_images[idx - len(self.masked_images)])

        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


Image Transformations

In [63]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize the images to 128x128
    transforms.ToTensor(),  # Convert image to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])


Load Dataset

In [64]:
# masked_dir = r'C:\Users\USER\OneDrive\Desktop\Masked Face Detection using Periocular Region\Masked_LFW_Dataset'  # Updated path
# unmasked_dir = r'C:\Users\USER\OneDrive\Desktop\Masked Face Detection using Periocular Region\LFW_without_Mask'  # Updated path
masked_dir = os.getcwd()+"\\Masked_LFW_Dataset\\Masked_LFW_Dataset"
unmasked_dir = os.getcwd()+"\\LFW_without_Mask\\LFW_without_Mask"
dataset = MaskedUnmaskedDataset(masked_dir, unmasked_dir, transform=transform)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Continue with your training loop...


Define CNN Model

In [65]:
model = models.resnet50(pretrained=True)

# Modify the final layer to match the number of classes in  dataset (Masked and Unmasked - 2 classes)
model.fc = torch.nn.Linear(in_features=model.fc.in_features, out_features=2)


In [66]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


 Training Loop
Here is the training loop that will run for a set number of epochs.

In [67]:
epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(data_loader)}")


Epoch [1/10], Loss: 0.02333869828317245
Epoch [2/10], Loss: 0.00013072862282203368
Epoch [3/10], Loss: 2.610697100855985e-05
Epoch [4/10], Loss: 1.4017465291051271e-05
Epoch [5/10], Loss: 2.325133741428791e-05
Epoch [6/10], Loss: 1.2494786265166084e-05
Epoch [7/10], Loss: 7.845057712647617e-06
Epoch [8/10], Loss: 6.376455188316424e-06
Epoch [9/10], Loss: 1.0620361658594489e-05
Epoch [10/10], Loss: 5.611205233391547e-06


Understanding the Loss Values
Loss Function:

The loss function measures how well the model's predictions match the true labels. Lower loss values indicate better performance.
In your case, you're using Cross-Entropy Loss, which is appropriate for multi-class classification problems (like identifying masked vs. unmasked faces).
Epoch Results:

Epoch 1: Loss starts at approximately 0.0233. This is a relatively low value, indicating that the model is already making good predictions early on.
Subsequent Epochs: The loss decreases rapidly over the first few epochs, indicating that the model is learning effectively. By Epoch 10, the loss is around 5.61e-06, which is extremely low.
Convergence: The rapid drop in loss suggests that the model has quickly learned the distinctions in the dataset. After the initial epochs, the decrease in loss is very gradual, indicating that the model may be nearing convergence (i.e., it has learned most of what it can from the training data).
Interpretation of Results
Overfitting Concerns: Given the very low loss values, it's possible that the model might be overfitting the training data, especially if your dataset is small. Overfitting occurs when the model learns to memorize the training data instead of generalizing to unseen data. This can be a concern if you see a significant drop in performance when testing on validation or test datasets.



Model Evaluation
After training, i evaluated the model using the evaluation code you provided. This part calculates the model's accuracy on the dataset, giving insight into its performance:



In [71]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in data_loader:  # You can use a separate test loader here
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy of the model: {100 * correct / total}%")


Accuracy of the model: 100.0%


In [75]:
print = 'k'